In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import numpy as np

c:\Users\Daksh Vashist\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
sp500_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_constituents.drop(sp500_constituents.columns[[1, 2, 3, 4, 5, 6, 7]], axis=1, inplace=True)

In [13]:
sp500_list = sp500_constituents['Symbol'].to_list()

sp500_list.remove('BRK.B')
sp500_list.remove('BF.B')

In [4]:
final_df = pd.DataFrame({'Stock Code': sp500_list,
                             'Momentum Score (C)': np.nan})

In [5]:
start = dt.datetime(2019, 1, 1)
end = dt.datetime(2023, 12, 31)

In [6]:
# Creating an empty dataframe for all of the stocks of S&P 500
sp500_histdata = pd.DataFrame({
    'Date':pd.date_range(start, end)
    }
)
#sp500_histdata[sp500_list] = np.nan

In [7]:
sp500_histdata.tail()

,Date
1821,2023-12-27
1822,2023-12-28
1823,2023-12-29
1824,2023-12-30
1825,2023-12-31


In [10]:
# Data extraction using STOOQ

name = 'AMZN'
df = web.DataReader(name, 'stooq', start, end)
df.rename(columns={'Close':name}, inplace=True)
df.reset_index(inplace=True)
#df.drop(df.columns[[1, 2, 3, 5]], axis=1, inplace=True)

df.head()

,Exceeded the daily hits limit


In [8]:
# Data extraction using YFINANCE

amazon = yf.Ticker('AMZN')
amazon_data = amazon.history(period='5y')
amazon_data.reset_index(inplace=True)
amazon_data.drop(amazon_data.columns[[1, 2, 3, 5, 6, 7]], axis=1, inplace=True)

In [12]:
 # Download stock close prices of a list of stocks using yfinance

for stock in sp500_list:
    try:
        df = yf.Ticker(stock).history(period='5y')
        df.rename(columns={'Close':stock}, inplace=True)
        df.reset_index(inplace=True)
        df.drop(df.columns[[1, 2, 3, 5, 6, 7]], axis=1, inplace=True)
        df.to_csv(stock + '.csv')
    except:
        pass

BRK.B: No data found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (period=5y)


In [25]:
df = pd.read_csv(sp500_list[0] + '.csv')
df.style.format({"Date": lambda t: t.strftime("%m/%d/%Y")})
df.head()

,Unnamed: 0,Date,MMM
0,0,2019-02-04 00:00:00-05:00,163.726334
1,1,2019-02-05 00:00:00-05:00,164.470474
2,2,2019-02-06 00:00:00-05:00,165.656250
3,3,2019-02-07 00:00:00-05:00,163.423737
4,4,2019-02-08 00:00:00-05:00,163.407394
